Q1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [34]:
# Importing Libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import selenium
from selenium import webdriver
import time
import re
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from datetime import datetime

In [59]:
driver = webdriver.Chrome(r"D://Chromedriver.exe")


In [60]:
url = 'https://www.amazon.in/'


In [61]:
#Opeaning the web page through our web driver:
driver.get(url)

In [62]:
#User Input:
user_inp = input('Enter the product you want to search : ')

Enter the product you want to search : Guitar


In [63]:
#Selecting Search Options:
search_bar = driver.find_element_by_id("twotabsearchtextbox")    
search_bar.clear()                                              
search_bar.send_keys(user_inp)                                  
search_button = driver.find_element_by_xpath('//div[@class="nav-search-submit nav-sprite"]/span/input')      
search_button.click()

In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [64]:
#Clicking the search button to get the results of "Input Keyword"
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//span[@id='nav-search-submit-text']"))).click()

#Getting the "href" value/URL of the product in first 3 pages
URT1=[]
for i in range(0, 3):
    for i in driver.find_elements_by_xpath("//a[@class='a-link-normal a-text-normal']"):
        URT1.append(i.get_attribute('href'))

    driver.find_element_by_xpath("//li[@class='a-last']").click()
    time.sleep(3)
        
#Lets create required empty lists required for the 
Avg_Ratings=[]
Brand=[]
No_of_Ratings=[]
Price=[]
Return_Exchange=[]
Expected_Delivery=[]
Availability=[]
Other_Details=[]
Product_URL=[]

#loop for all the required data of product
for url in URT1:
    driver.get(url)
    
#To find the Avg_Ratings of the product
    try:                  
        rate=driver.find_element_by_xpath("//span[@id='acrCustomerReviewText']")
        rate.click()                                                      
        rating=driver.find_element_by_xpath("//span[@class='a-size-medium a-color-base']")
        Avg_Ratings.append(rating.text)
        
    except NoSuchElementException   as e:
        Avg_Ratings.append("-")
               
#To find the Brand Name & Details of the product
    try: 
        brand=driver.find_element_by_xpath("//span[@id='productTitle']")
        Brand.append(brand.text)
        
    except NoSuchElementException   as e:
        Brand.append("-")       
        
#To find the Brand Name & Details of the product  
    try: 
        ratings=driver.find_element_by_xpath("//span[@class='a-size-base a-color-secondary']")
        No_of_Ratings.append(ratings.text)
        
    except NoSuchElementException   as e:
        No_of_Ratings.append("-")
        
#To find the Price of the product 
    try: 
        price=driver.find_element_by_xpath("//td[@class='a-span12']")
        Price.append(price.text)
        
    except NoSuchElementException   as e:
        Price.append("-")
              
#To find the Return & Exchange policy of the product 
    try: 
        
        ret_exchange=driver.find_element_by_xpath("//div[@class='a-column a-span3 a-text-center icon-container  icon-grid-width']")#locating the brand
        Return_Exchange.append(ret_exchange.text)
        
    except NoSuchElementException   as e:
        Return_Exchange.append("-")
              
#To find the Expected Delivery policy of the product 
    try: 
        
        Delivery=driver.find_element_by_xpath("//div[@id='dynamicDeliveryMessage']")
        Expected_Delivery.append(Delivery.text)
        
    except NoSuchElementException   as e:
        Expected_Delivery.append("-")        
        
#To find the Availability of the product 
    try: 
        
        availability=driver.find_element_by_xpath("//span[@class='a-size-medium a-color-success']")
        Availability.append(availability.text)
        
    except NoSuchElementException   as e:
        Availability.append("-")       
        
#To find the Other Details of the product
    try: 
        
        Details=driver.find_element_by_xpath("//div[@id='feature-bullets']")
        Other_Details.append(Details.text)
        
    except NoSuchElementException   as e:
        Other_Details.append("-")
        
#Creating the dataframe for the Searched Keyword/Product searched on Amazon

DF=pd.DataFrame({})
DF['Brand']=Brand
DF['No_of_Ratings']=No_of_Ratings
DF['Avg_Ratings']=Avg_Ratings
DF['Price']=Price
DF['Expected_Delivery']=Expected_Delivery
DF['Other Details']=Other_Details


#Creating the dataframe for Product_Vertical
Product_Vertical=pd.DataFrame({})

#Brand Name of the Guitar
Product_Vertical['Brand Name'] = DF['Brand'].str.split(' ').str[0]

#Name Of Product/Guitar mentioned on Amazon
Product_Vertical['Name Of Product'] = DF['Brand'].str.split(' ').str[1:]
Product_Vertical['Name Of Product'] = DF['Brand'].str.replace('[.,(),]', '')

#Average Ratings for the Guitar 
Product_Vertical['Average Rating']=DF['Avg_Ratings'].str.split(' ').str[0]

#No_of_Ratings for the Guitar 
Product_Vertical['No Of Ratings'] = DF['No_of_Ratings'].str.split(' ').str[0]

#Price of the Guitar
Product_Vertical['Price_INR'] = DF['Price'].str.split(' ').str[1:2]

#Return policy of product
Product_Vertical['Return Policy'] = Return_Exchange

#Expected Delivery of product
Product_Vertical['Expected Delivery'] = DF['Expected_Delivery'].str.split(' ').str[2:5].replace('[.,(),]', '')

#Availability of product
Product_Vertical['Product Availability'] = Availability

#Other Details of product
Product_Vertical['Other Details'] = DF['Other Details'].str.replace('\n', ' ')

#URL of product
Product_Vertical['Product URL'] = URT1

print('The DataFrame for Searched product on https://www.amazon.in/ is :')

print("\n")
Product_Vertical

The DataFrame for Searched product on https://www.amazon.in/ is :




,Brand Name,Name Of Product,Average Rating,No Of Ratings,Price_INR,Return Policy,Expected Delivery,Product Availability,Other Details,Product URL
0,Kadence,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,4,404,"[5,999.00]",-,"[Saturday,, May, 8]",In stock.,About this item QUALITY STRINGS ---The surface...,https://www.amazon.in/gp/slredirect/picassoRed...
1,Kadence,Kadence Slowhand Premium Jumbo Semi Acoustic G...,4.3,107,"[9,999.00]",-,"[Saturday,, May, 8]",In stock.,QUALITY STRINGS ---The surface is coated with ...,https://www.amazon.in/gp/slredirect/picassoRed...
2,GUITAR,GUITAR BRO - COMBO Blue Acoustic Guitar for Be...,4.1,18,"[6,499.00]",-,"[May, 7, -]",In stock.,About this item GUITAR BRO +20 mins FREE VIDEO...,https://www.amazon.in/gp/slredirect/picassoRed...
3,Divyatamall,Divyatamall 22 inches 4 String Wooden Finish A...,3.5,2,[787.00],-,"[Tuesday,, May, 11]",In stock.,About this item Dreadnought body style Spruce ...,https://www.amazon.in/gp/slredirect/picassoRed...
4,Juârez,Juârez Acoustic Guitar 38 Inch Cutaway 038C wi...,3.9,"9,184","[2,290.00]",-,"[Monday,, May, 10]",In stock.,"About this item Black Glossy Finish, Number of...",https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...,...,...,...
157,DARK,"DARK Seller Toy's & Gift 23"" 4 String Decor Gu...",-,-,[899.00],-,"[delivery:, Tuesday,, May]",In stock.,About this item Very Crisp and Clear Sound of ...,https://www.amazon.in/DARK-Childrens-Instrumen...
158,QUALITYZONE,QUALITYZONE 4-String Musical Instrument Guitar...,-,-,"[1,595.00]",-,"[Tuesday,, May, 11]",In stock.,About this item Very Crisp and Clear Sound of ...,https://www.amazon.in/QUALITYZONE-4-String-Ins...
159,Juârez,Juârez JRZ38C/ZEB 6 Strings Acoustic Guitar 38...,4,62,"[2,949.00]",-,"[Sunday,, May, 9]",In stock.,"About this item Number of Frets: 18, Acoustic ...",https://www.amazon.in/Juarez-Strings-Acoustic-...
160,Hawaii,Hawaii Kids Guitar Soprano Ukulele Durable Fle...,-,-,"[2,339.00]",-,"[delivery:, May, 21]",In stock.,About this item The natural and unrestrained p...,https://www.amazon.in/Soprano-Ukulele-Flexible...


Q3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.




In [24]:
#importing all the required libraries
import pandas as pd
import selenium
from selenium import webdriver
import re
import requests
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

In [66]:
#connect to the web driver
driver = webdriver.Chrome(r"D://Chromedriver.exe")
#Specifying the url of the webpage
driver.get("https://images.google.com/")

In [67]:
#Assigning the "Search_Keys" option to search the Specific Keyword on Google
Search_Keys = input("> Type the Keyword to Search on https://images.google.com/  :----> ")

#Finding the "Assigned Keyword (Search_Keys)" on Google
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//input[@class='gLFyf gsfi']"))).send_keys(Search_Keys)

#Clicking the search button to get the results of "Input Keyword"
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//span[@class='z1asCe MZy1Rb']"))).click()

> Type the Keyword to Search on https://images.google.com/  :----> Fruits


In [68]:
#Getting the "src" image value of the word on Google
URT1=[]
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
        URT1.append(i.get_attribute('src'))
        
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    URT1.append(i.get_attribute('data-src'))
    
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    URT1.append(i.get_attribute('alt'))


In [70]:
#Dataframe for Fruit_Images
Fruit_Images=[]
Fruit_Images=URT1

In [71]:
#Checking the length
print('> Length of Fruit_Images :', len(Fruit_Images))
print("\n")

> Length of Fruit_Images : 147




In [72]:
#connect to the web driver
driver=webdriver.Chrome(r"D://Chromedriver.exe")
#Specifying the url of the webpage
driver.get("https://images.google.com/")


In [73]:
#Assigning the "Search_Keys" option to search the Specific Keyword on Google
Search_Keys = input("> Type the Keyword to Search on https://images.google.com/  :----> ")

#Finding the "Assigned Keyword (Search_Keys)" on Google
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//input[@class='gLFyf gsfi']"))).send_keys(Search_Keys)

#Clicking the search button to get the results of "Input Keyword"
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//span[@class='z1asCe MZy1Rb']"))).click()


#Getting the "src" image value of the word on Google
URT2=[]
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
        URT2.append(i.get_attribute('src'))
        
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    URT2.append(i.get_attribute('data-src'))
    
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    URT2.append(i.get_attribute('alt'))


#Dataframe for Fruit_Images
Car_Images=[]
Car_Images=URT2

#Checking the length
print('> Length of Car_Images :', len(Car_Images))
print("\n")

> Type the Keyword to Search on https://images.google.com/  :----> Car
> Length of Car_Images : 147




In [74]:
#connect to the web driver
driver=webdriver.Chrome(r"D://Chromedriver.exe")
#Specifying the url of the webpage
driver.get("https://images.google.com/")

In [75]:
#Assigning the "Search_Keys" option to search the Specific Keyword on Google
Search_Keys = input("> Type the Keyword to Search on https://images.google.com/  :----> ")

#Finding the "Assigned Keyword (Search_Keys)" on Google
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//input[@class='gLFyf gsfi']"))).send_keys(Search_Keys)

#Clicking the search button to get the results of "Input Keyword"
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//span[@class='z1asCe MZy1Rb']"))).click()


#Getting the "src" image value of the word on Google
URT3=[]
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
        URT3.append(i.get_attribute('src'))
        
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    URT3.append(i.get_attribute('data-src'))
    
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    URT3.append(i.get_attribute('alt'))


#Dataframe for Fruit_Images
ML_Images=[]
ML_Images=URT3

#Checking the length
print('> Length of Machine_Learning_Images :', len(ML_Images))
print("\n")

> Type the Keyword to Search on https://images.google.com/  :----> Machine learning
> Length of Machine_Learning_Images : 144




In [76]:
#Putting all in a single dataframe
Images=pd.DataFrame({})
Images['Fruit_Images']=Fruit_Images[0:100]
Images['Car_Images']=Car_Images[0:100]
Images['Machine_Learning_Images']=ML_Images[0:100]
Images

,Fruit_Images,Car_Images,Machine_Learning_Images
0,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA..."
3,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
4,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
...,...,...,...
95,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
96,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,Why Machine Learning Needs Semantics Not Just ...
97,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,What You Need To Know About Machine Learning
98,Buy Fruits (All in One) Book Online at Low Pri...,Car of the Year: ExtremeTech's Best Cars for 2...,How Machine Learning Optimizes the Supply Chai...



Question 4
Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”,“RAM” , “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [41]:
#connect to the web driver
driver=webdriver.Chrome(r"D://Chromedriver.exe")
driver.get('https://www.flipkart.com/')

In [42]:
product=input('Enter the Product to be searched ')
driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]').click()
time.sleep(3)
driver.find_element_by_xpath('//input[@title="Search for products, brands and more"]').send_keys(str(product))
driver.find_element_by_xpath('//button[@type="submit"]').click()
time.sleep(4)

Enter the Product to be searched pixel 4A


In [44]:
urls=[]
brand=[]
name=[]
color=[]
RAM=[]
ROM=[]
prim_cam=[]
sec_cam=[]
display_size=[]
display_res=[]
processor=[]
processor_core=[]
battery=[]
price=[]
links=driver.find_elements_by_xpath('//a[@class="_1fQZEK"]')

In [47]:
for i in links:
    urls.append(i.get_attribute('href'))
for i in urls:
    driver.get(i)
    time.sleep(3)
    driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]').click()
    try:
        title=driver.find_element_by_xpath('//span[@class="B_NuCI"]').text.split()
        brand.append(title[0])
    except:
        brand.append('-')
    try:
        name.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[1]/table/tbody/tr[3]/td[2]').text)
    except:
        name.append('-')
    try:
        color.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[1]/table/tbody/tr[4]/td[2]').text)
    except:
        color.append('-')
    try:
        RAM.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[2]/td[2]').text)
    except:
        RAM.append('-')
    try:
        ROM.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[1]/td[2]').text)
    except:
        ROM.append('-')
    try:
        prim_cam.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[2]/td[2]').text)
    except:
        prim_cam.append('-')
    try:
        sec_cam.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[5]/td[2]').text)
    except:
        sec_cam.append('-')
    try:
        display_size.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[1]/td[2]').text)
    except:
        display_size.append('-')
    try:
        display_res.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[2]/td[2]').text)
    except:
        display_res.append('-')
    try:
        processor.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[3]/table/tbody/tr[2]/td[2]').text)
    except:
        processor.append('-')
    try:
        processor_core.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[3]/table/tbody/tr[3]/td[2]').text)
    except:
        processor_core.append('-')
    try:
        battery.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[10]/table/tbody/tr[1]/td[2]').text)
    except:
        battery.append('-')
    try:
        price.append(driver.find_element_by_xpath('//div[@class="dyC4hf"]/div/div/div').text)
    except:
        price.append('-')
        
flip_search=pd.DataFrame({})
flip_search['brand']=brand
flip_search['name']=name
flip_search['color']=color
flip_search['RAM']=RAM
flip_search['ROM']=ROM
flip_search['prim_cam']=prim_cam
flip_search['sec_cam']=sec_cam
flip_search['display_size']=display_size
flip_search['display_res']=display_res
flip_search['processor']=processor
flip_search['processor_core']=processor_core
flip_search['battery']=battery
flip_search['price']=price
flip_search['urls']=urls
flip_search

,brand,name,color,RAM,ROM,prim_cam,sec_cam,display_size,display_res,processor,processor_core,battery,price,urls
0,Google,Pixel 4a,Just Black,6 GB,128 GB,12.2MP Rear Camera,8MP Front Camera,14.76 cm (5.81 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 730G,Octa Core,69.4 mm,"₹26,999",https://www.flipkart.com/google-pixel-4a-just-...
1,REDMI,Note 9,Arctic White,4 GB,64 GB,48MP Rear Camera,13MP AI Selfie Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,MediaTek Helio G85,Octa Core,76.7 mm,"₹10,999",https://www.flipkart.com/redmi-note-9-arctic-w...
2,REDMI,Note 9,Aqua Green,4 GB,64 GB,48MP Rear Camera,13MP AI Selfie Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,MediaTek Helio G85,Octa Core,76.7 mm,"₹10,999",https://www.flipkart.com/redmi-note-9-aqua-gre...
3,REDMI,9 Power,Blazing Blue,4 GB,64 GB,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"₹9,999",https://www.flipkart.com/redmi-9-power-blazing...
4,REDMI,Note 9,Pebble Grey,4 GB,64 GB,48MP Rear Camera,13MP AI Selfie Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,MediaTek Helio G85,Octa Core,76.7 mm,"₹10,999",https://www.flipkart.com/redmi-note-9-pebble-g...
5,REDMI,Note 9,Shadow Black,4 GB,64 GB,48MP Rear Camera,13MP AI Selfie Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,MediaTek Helio G85,Octa Core,76.7 mm,"₹10,999",https://www.flipkart.com/redmi-note-9-shadow-b...
6,REDMI,9 Power,Electric Green,4 GB,64 GB,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"₹9,999",https://www.flipkart.com/redmi-9-power-electri...
7,Redmi,Redmi 9A,Nature Green,2 GB,32 GB,13MP Rear Camera,Yes,16.59 cm (6.53 inch),720 x 1600$$pixel,Octa Core,2 GHz,1 Year Manufacturer Warranty,"₹7,181",https://www.flipkart.com/redmi-9a-nature-green...
8,Redmi,Redmi 9 Prime,Matte Black,4 GB,64 GB,13MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G80,Octa Core,5020 mAh,"₹9,499",https://www.flipkart.com/redmi-9-prime-matte-b...
9,REDMI,9 Power,Mighty Black,6 GB,128 GB,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,Qualcomm® Snapdragon™ 662,Octa Core,6000 mAh,"₹12,999",https://www.flipkart.com/redmi-9-power-mighty-...


In [48]:
# Saving into a csv file
flip_search.to_csv('flip_search.csv')


Question 5
Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [49]:
driver=webdriver.Chrome(r"D://Chromedriver.exe")
driver.get('https://www.google.com/maps/')

In [50]:
# selecting input field 
city = input('Enter City Name : ') 
driver.find_element_by_id('searchboxinput').send_keys(city)
# pressing Enter
driver.find_element_by_id('searchboxinput').send_keys(Keys.ENTER)
time.sleep(10)
#Scraping latitude and Longitude from page URL
lat=driver.current_url.split('/@')[1].split(',')[0]
long=driver.current_url.split('/@')[1].split(',')[1].split(',')[0]
print('Latitude :',lat,'Longitude :',long)

Enter City Name : Jaipur
Latitude : 26.8851417 Longitude : 75.6504688


Question 6.
Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [51]:
#connect to the web driver
driver=webdriver.Chrome(r"D://Chromedriver.exe")
driver.get('https://trak.in/')
button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(button)

In [52]:
# making directories
fund_dict = {}
fund_dict['Date'] = []
fund_dict['Startup Name'] = []
fund_dict['Industry/Vertical'] = []
fund_dict['Sub-Vertical'] = []
fund_dict['Location'] = []
fund_dict['Investor'] = []
fund_dict['Investment Type'] = []
fund_dict['Amount(in USD)'] = []

In [53]:
for i in range(48,51):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]'.format(i)).click()

    # Date
    dt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        fund_dict['Date'].append(d.text)

    # Startup Name
    sn = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        fund_dict['Startup Name'].append(n.text)
    
    # Industry/Vertical
    ind = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        fund_dict['Industry/Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        fund_dict['Sub-Vertical'].append(s.text)
        
    # Location
    loc = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        fund_dict['Location'].append(l.text)
    
    # Investor
    inv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        fund_dict['Investor'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        fund_dict['Investment Type'].append(n.text)
    
    # Amount
    amt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        fund_dict['Amount(in USD)'].append(a.text)
        
fund_df = pd.DataFrame(fund_dict)
fund_df

,Date,Startup Name,Industry/Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


Question 7¶Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [55]:
driver=webdriver.Chrome(r"D://Chromedriver.exe")
driver.get('https://www.digit.in/')

In [56]:
#Navigating to the link of best gaming laptop
l1 = driver.find_element_by_xpath("//div[@class='listing_container']/ul/li[9]/a").get_attribute('href')
driver.get(l1)

In [57]:
name = []
price = []
OS = []
display = []
processor = []
HDD = []
RAM = []
weight = []
dimension = []
GPU = []

#names
names=driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3")
for i in names:
    name.append(i.text)
    
#os
os=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[1]/div/div")
for i in os:
    OS.append(i.text)
    
#display
displays=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[2]/div/div")
for i in displays:
    display.append(i.text)
    
#processor
processors=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[3]/div/div")
for i in processors:
    processor.append(i.text)
processor

#memory
memories=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[1]")#list of specificaion name
memories_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]")#values of specifiations 
for i in range(len(memories)):
        if memories[i].text=='Memory':
            HDD.append(memories_spec[i].text.split('/')[0])
            RAM.append(memories_spec[i].text.split('/')[1])
        else:
            HDD.append('No details available')#append no details as value for memory is missing in some of the laptops
            RAM.append('No details available')#append no details as value for memory is missing in some of the laptops

#weight
weights=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")#list of specificaion name
weight_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")#values of specifiations
for i in range(len(weights)):
        if weights[i].text=='Weight':
            weight.append(weight_spec[i].text)
            
#dimension
dimension=[]
dims=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")#list of specificaion name
dims_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")#values of specifiations
for i in range(len(dims)):
        if dims[i].text=='Dimension':
            dimension.append(dims_spec[i].text)

#graphical processor
GPUs=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[1]")#list of specificaion name
GPUs_spec=driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr/td[3]")#values of specifiations
for i in range(len(GPUs)):
        if GPUs[i].text=='Graphics Processor':
            GPU.append(GPUs_spec[i].text)
        
full_specs=[]
urls=driver.find_elements_by_xpath("//div[@class='full-specs']/span")#getting the url of full specs links
for i in urls:
    if i.get_attribute('data-href'):
        full_specs.append(i.get_attribute('data-href'))
        
for i in full_specs:#iterating throug every laptops full specs' page
    driver.get(i)
    try:
        prices=driver.find_element_by_xpath("//div[@class='Block-price']/b")
        price.append(prices.text)
    except NoSuchElementException:#exception handling for no price details
        price.append("No details available")
        
df=pd.DataFrame({"Name":name,
                "Price":price,
                "OS":OS,
                "Display":display,
                "HDD":HDD,
                 "RAM":RAM,
                "processor":processor,
                "weight":weight,
                "Dimension":dimension,
                "Graphical processor":GPU})
df

,Name,Price,OS,Display,HDD,RAM,processor,weight,Dimension,Graphical processor
0,ALIENWARE AREA 51M R2,No details available,WINDOWS 10 HOME,"17.3"" (1920 X 1080)",1 TB SSD,16 GBGB DDR4,10TH GENERATION INTEL® CORE™ I7-10700 | 2.90 GHZ,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630
1,ALIENWARE M15 R3,"341,990",WINDOWS 10 HOME,"15.6"" (3840 X 2160)",1 TB SSD,16 GBGB DDR4,10TH GENERATION INTEL® CORE™ I9-10980HK | NA,NA,NA,NA
2,ASUS ROG STRIX SCAR 15,No details available,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",1 TB SSD,16 GBGB DDR4,AMD RYZEN™ 9 5900HX | 3.3 GHZ,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070
3,ASUS ROG ZEPHYRUS G14,"164,990",WINDOWS 10 HOME,"14"" (1920 X 1080)",1 TB SSD,16 GBGB DDR4,AMD 3RD GENERATION RYZEN 9 | 3.3 GHZ,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060
4,LENOVO LEGION 5I,"71,990",WINDOWS 10 PRO,"15.6"" (1920 X 1080)",1 TB SSD,16 GBGB DDR4,10TH GENERATION INTEL® CORE™ I5-10300H | 2.50 GHZ,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB
5,ASUS ROG ZEPHYRUS DUO 15,"244,990",WINDOWS 10,"15.6"" (3840 X 1100)",512 GB SSD,4 GBGB DDR4,INTEL CORE I7 10TH GEN 10875H | NA,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q
6,ACER ASPIRE 7 GAMING,"56,990",WINDOWS 10 HOME,"15.6"" (1920 X 1080)",512 GB SSD,8 GBGB DDR4,AMD RYZEN™ 5-5500U HEXA-CORE | NA,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650
